In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

<h1 style="font-family:arial;"> <center> Using CNN to predict dog or cat in any image </center> </h1>
<h3><center>The main goal of this tutorial is to develop a system that can identify images of cats and dogs.</h3></center>

# Introduction

In this notebook, we will learn how to classify images of dog and cat by building a simple Neural Network. What we will learn:

- Load the image
- Visualize the Data distribution of all data
- Visualizing some of the images
- How CNN works
- Use Image Data Generator
- Graph the training loss and validation loss
- Model  Evaluation
- Confusion Matrix

In [2]:
# Importing the libraries

import zipfile
import os
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.image as img
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

In [4]:
'''setting seed'''
seed = 0
np.random.seed(seed)
tf.random.set_seed(3)

In [5]:
import zipfile

zip_files = ['test1', 'train']

for zip_file in zip_files:
    with zipfile.ZipFile("../input/dogs-vs-cats/{}.zip".format(zip_file),"r") as z:
        z.extractall(".")
        print("{} unzipped".format(zip_file))

In [6]:
print(os.listdir('../working'))

# Importing Data

In [7]:
IMAGE_FOLDER_PATH = "../working/train"
FILE_NAMES = os.listdir(IMAGE_FOLDER_PATH)
WIDTH = 150
HEIGHT = 150

In [8]:
FILE_NAMES[0:5]

In [9]:
labels = []
for i in os.listdir(IMAGE_FOLDER_PATH):
    labels+=[i]

# Data distribuation of all data

In [10]:
# empty list
targets = list()
full_paths = list()
train_cats_dir = list()
train_dogs_dir = list()

# finding each file's target
for file_name in FILE_NAMES:
    target = file_name.split(".")[0] # target name
    full_path = os.path.join(IMAGE_FOLDER_PATH, file_name)
    
    if(target == "dog"):
        train_dogs_dir.append(full_path)
    if(target == "cat"):
        train_cats_dir.append(full_path)
    
    full_paths.append(full_path)
    targets.append(target)

dataset = pd.DataFrame() # make dataframe
dataset['image_path'] = full_paths # file path
dataset['target'] = targets # file's target

In [11]:
dataset

In [12]:
print("total data counts:", dataset['target'].count())
counts = dataset['target'].value_counts()
print(counts)

# EDA

In [13]:
fig = go.Figure(go.Bar(
            x= counts.values,
            y=counts.index,
            orientation='h'))

fig.update_layout(title='Data Distribution in Bars',font_size=15,title_x=0.45)
fig.show()

In [14]:
fig=px.pie(counts.head(10),values= 'target', names=dataset['target'].unique(),hole=0.425)
fig.update_layout(title='Data Distribution of Data',font_size=15,title_x=0.45,annotations=[dict(text='Cat vs Dog',font_size=18, showarrow=False,height=800,width=700)])
fig.update_traces(textfont_size=15,textinfo='percent')
fig.show()

# Displaying Images of Cat

In [15]:
rows = 4
cols = 4
axes = []
fig=plt.figure(figsize=(10,10))
i = 0

for a in range(rows*cols):
    b = img.imread(train_cats_dir[i])
    axes.append(fig.add_subplot(rows,cols,a+1))
    plt.imshow(b)
    i+=1
fig.tight_layout()
plt.show()

# Displaying Images of Dog

In [16]:
rows = 4
cols = 4
axes = []
fig=plt.figure(figsize=(10,10))
i = 0

for a in range(rows*cols):
    b = img.imread(train_dogs_dir[i])
    axes.append(fig.add_subplot(rows,cols,a+1))
    plt.imshow(b)
    i+=1
fig.tight_layout()
plt.show()

# Splitting the data into train and test data

In [17]:
dataset_train, dataset_test = train_test_split(dataset, test_size=0.2, random_state=seed)

# Data Distribution of Train Data 

In [18]:
class_id_distributionTrain = dataset_train['target'].value_counts()
class_id_distributionTrain

In [19]:
fig = go.Figure(go.Bar(
            x=class_id_distributionTrain.values,
            y=class_id_distributionTrain.index,
            orientation='h'))

fig.update_layout(title='Data Distribution Of Train Data in Bars',font_size=15,title_x=0.45)
fig.show()

In [20]:
fig=px.pie(class_id_distributionTrain.head(10),values= 'target', names=dataset_train['target'].unique(),hole=0.425)
fig.update_layout(title='Data Distribution of Train Data in Pie Chart',font_size=15,title_x=0.45,annotations=[dict(text='Cat vs Dog',font_size=18, showarrow=False,height=800,width=700)])
fig.update_traces(textfont_size=15,textinfo='percent')
fig.show()

# Data Distribution of Test data

In [21]:
class_id_distributionTest = dataset_test['target'].value_counts()
class_id_distributionTest

In [22]:
fig = go.Figure(go.Bar(
            x=class_id_distributionTest.values,
            y=class_id_distributionTest.index,
            orientation='h'))

fig.update_layout(title='Data Distribution Of Train Data in Bars',font_size=15,title_x=0.45)
fig.show()

In [23]:
fig=px.pie(class_id_distributionTrain.head(10),values= 'target', names=dataset_train['target'].unique(),hole=0.425)
fig.update_layout(title='Data Distribution of Train Data in Pie Chart',font_size=15,title_x=0.45,annotations=[dict(text='Cat vs Dog',font_size=18, showarrow=False,height=800,width=700)])
fig.update_traces(textfont_size=15,textinfo='percent')
fig.show()

# Now, we will use ImageDataGenerator
When there is little data to train, we have to use ImageDataGenerator to increase the number of data.
- rescale = 1./255 : change the value between 0 and 1
- rotation_range = 15 : Random rotation within 15 degrees
- shear_range = 0.1 : shear range 10%
- zoom_range = 0.2 : zoom range 20%
- horizontal_flip = True : Randomly flip horizontally
- width_shift_range = 0.1 : Randomly move the original image horizontally within 10% of the width
- height_shift_range=0.1 : Randomly move the original image vertically within 10% of the width

In [24]:
train_datagen=ImageDataGenerator(
rotation_range=15,
rescale=1./255,
shear_range=0.1,
zoom_range=0.2,
horizontal_flip=True,
width_shift_range=0.1,
height_shift_range=0.1)

In [25]:
train_datagenerator=train_datagen.flow_from_dataframe(dataframe=dataset_train,
                                                     x_col="image_path",
                                                     y_col="target",
                                                     target_size=(WIDTH, HEIGHT),
                                                     class_mode="binary",
                                                     batch_size=150)

In [26]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_datagenerator=test_datagen.flow_from_dataframe(dataframe=dataset_test,
                                                   x_col="image_path",
                                                   y_col="target",
                                                   target_size=(WIDTH, HEIGHT),
                                                   class_mode="binary",
                                                   batch_size=150)

# Making CNN Model

![CNN](https://raw.githubusercontent.com/AayushSaxena08/Image_Classification_using_Transfer_Learning/main/Cats_Dogs_CNN.gif)

### The Convolutional Neural Network is made up of 3 parts:
- Convolutional Layer
- Pooling Layer
- Fully Connected Layer

![CNN Architecture](https://github.com/AayushSaxena08/Image_Classification_using_Transfer_Learning/blob/main/cnn1.jpeg?raw=true)

### 1. Convolutional layer
A convolutional layer helps to extract the information from the image with the help of filter(kernal).Please have a look the following image.
![CNN](https://github.com/AayushSaxena08/Image_Classification_using_Transfer_Learning/blob/main/Filter.gif?raw=true)

#### Feature Map
- It will Take filter and map into image into size of 3 *3.
- Then It will do some mathematical operation.
- The Elements get Multiplied and then added to get one output and that out put will save in output feature map
- Below Image will clearfiy alot of thing

### 2. Pooling Layer

- The Intitution behind the pooling layer is to reduce the dimension of feature map.
- The 2*2 pooling layer will apply to the  input image and take the maximum value at each slide
- Following Image will clear the concept more easily

![Pooling_Layer](https://github.com/AayushSaxena08/Image_Classification_using_Transfer_Learning/blob/main/pooling_layer.gif?raw=true)

### 3. Fully Connected Layer
- The final layer is fully connected layer
- The input to the fully connected layer is the rich features that have been extracted using convolutional filters.
- This will follow to the output layer where the probablities will be calculated of the image.

# Compile all of three

In [27]:
# Compile the model

model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), input_shape=(WIDTH, HEIGHT, 3), activation='relu'))
model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

In [28]:
model.summary()

![Final_CNN_Architecture](https://github.com/AayushSaxena08/Image_Classification_using_Transfer_Learning/blob/main/CNN_final.PNG?raw=true)
<h3><center>Final architecture of CNN</center></h3>

# Displaying The Model

In [29]:
from tensorflow.keras.utils import plot_model
from IPython.display import Image
plot_model(model, to_file='convnet.png', show_shapes=True,show_layer_names=True)
Image(filename='convnet.png') 

In [30]:
!pip install visualkeras

In [31]:
import visualkeras
visualkeras.layered_view(model, legend=True) 

# Train The Model

In [32]:
History=model.fit(train_datagenerator,
                  epochs=10,
                  validation_data=test_datagenerator,
                  validation_steps=dataset_test.shape[0]/150,
                  steps_per_epoch=dataset_train.shape[0]/150)

# Plotting Loss and Accuracy

In [33]:
acc = History.history['accuracy']
val_acc = History.history['val_accuracy']
loss = History.history['loss']
val_loss = History.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'go', label='Training Loss')
plt.plot(epochs, val_loss, 'g', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# Model Evaluation

In [34]:
 test_loss, test_acc = model.evaluate(test_datagenerator, steps=len(test_datagenerator), verbose=1)
 print('Loss: %.3f' % (test_loss * 100.0))
 print('Accuracy: %.3f' % (test_acc * 100.0)) 

# Confusion Matrix

In [35]:
from sklearn.metrics import confusion_matrix
import itertools

In [36]:
predictions = model.predict(x=test_datagenerator, steps= len(test_datagenerator), verbose=0)

In [37]:
test_datagenerator.classes

In [38]:
cm = confusion_matrix(y_true=test_datagenerator.classes, y_pred=np.argmax(predictions, axis=-1))

In [39]:
def plot_confusion_matrix(cm, classes,normalize=False,title='Confusion matrix',cmap=plt.cm.Blues):
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
        print(cm)
    
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
        horizontalalignment="center",
        color="white" if cm[i, j] > thresh else "black")
        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')

In [40]:
cm_plot_labels = ['0_cat', '1_dog']

plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

# Please Upvote 👍